In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import time


base_url='https://makanilebanon.com/'

def getCategories():
     html_text = requests.get('https://makanilebanon.com/portal/en-US/directory/').text
     soup = BeautifulSoup(html_text, 'lxml')
     categories = soup.find_all('div', class_='menu-item')
     list_of_categories=[]
     for category in categories:
          list_of_categories.append(category.a['href'])
     list_of_categories.pop()
     list_of_categories.pop()
     return list_of_categories

def getSubCategories():
     list_category_links=getCategories()
     list_of_subcategories_links=[]
     for category_link in list_category_links:
           html_text = requests.get(base_url+category_link).text
           soup = BeautifulSoup(html_text, 'lxml')
           subcategories= soup.find_all('div', class_='category-item')
           for subcategory in subcategories:
                if subcategory.a['href'] == '#':
                     sub_subcategories=subcategory.find_all('div', class_='sub-category-item')                    
                     for sub_subcategory in sub_subcategories:
                          list_of_subcategories_links.append(sub_subcategory.a['href'])
                else:
                     list_of_subcategories_links.append(subcategory.a['href'])
     return list_of_subcategories_links

def getSubcategoryTotalShops():
     list_of_subcategories_links=getSubCategories()
     list_of_subcategories=[]
     for subcategory_link in list_of_subcategories_links:
          html_text = requests.get(base_url+subcategory_link).text
          soup = BeautifulSoup(html_text, 'lxml')
          subcategory_name=subcategory_link.split('/')[4].replace('-', ' ').capitalize()       
          paging_info = soup.find('div', class_='paging-info').text.strip()
          total = paging_info.split('Total:')[1].strip().split()[0]
          total = total.replace('(', '').replace(')', '')
          s={
               "subcategory_name":subcategory_name,
               "total":int(total)
          }
          list_of_subcategories.append(s)
     return list_of_subcategories
             
     


def getShops():
    list_of_subcategories_links = getSubCategories()
    list_of_shops = []
    for subcategory_link in list_of_subcategories_links:
        html_text = requests.get(base_url + subcategory_link).text
        soup = BeautifulSoup(html_text, 'lxml')
        paging_info = soup.find('div', class_='paging-info').text.strip()
        nb_of_pages = int(paging_info.split('of')[1].strip().split()[0])
        for page_nb in range(1, nb_of_pages + 1):
            html_text_single_page = requests.get(f"{base_url}{subcategory_link}/page/{page_nb}").text
            soup = BeautifulSoup(html_text_single_page, 'lxml')
            shops = soup.find_all('div', class_='category-item post-item')
            for shop in shops:
                shop_name = shop.find('span', class_='title').text
                shop_phone_number = shop.find('span', class_='phone').text
                fullAddress = shop.find('span', class_='post-address').text.strip().split('>')
                city = fullAddress[0]
                location = fullAddress[1]
                shop_info = shop.find('span', class_='post-category').text.strip().split('>')
                if len(shop_info) == 2:
                    category = shop_info[0]
                    subcategory = shop_info[1]
                    sub_subcategory = ''
                else:
                    category = shop_info[0]
                    subcategory = shop_info[1]
                    sub_subcategory = shop_info[2]
                s = {
                    "shop_name": shop_name,
                    "phone_number": shop_phone_number,
                    "category": category,
                    "subcategory": subcategory,
                    "sub_subcategory": sub_subcategory,
                    "city": city,
                    "location": location
                }
                print()
                list_of_shops.append(s)
                print(s)  # Print the scraped shop data immediately after scraping it
    return list_of_shops
                      
                          
header = ['shop_name', 'phone_number', 'category', 'subcategory', 'sub_subcategory','city','location']

def writeToFile():
    with open('shops.csv', 'w', newline='', encoding='UTF8') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        shops = getShops()
        for shop in shops:
            writer.writerow(shop)
        print('saved to shops.csv file')

subcategoryHeaders=['subcategory_name','total']
def writeSubcategoriesFile():
    with open('Subcategories.csv', 'w', newline='', encoding='UTF8') as f:
        writer = csv.DictWriter(f, fieldnames=subcategoryHeaders)
        writer.writeheader()
        subcategories=getSubcategoryTotalShops()
        for subcategory in subcategories:
            writer.writerow(subcategory)
        print('saved to Subcategories.csv file')



writeToFile()
writeSubcategoriesFile()

In [9]:
import pandas as pd
df = pd.read_csv('shops.csv')
df.drop(columns=['sub_subcategory'], inplace=True)
df.head()

,shop_name,phone_number,category,subcategory,city,location
0,Abdullah Exhibition,03-773813,Vehicles & Transport,Agencies & Car Showrooms,South Governorate,Sidon District
1,AGA Classic Cars,03-514335,Vehicles & Transport,Agencies & Car Showrooms,North Governorate,City of Tripoli
2,Al Haytham Expo,71-582907,Vehicles & Transport,Agencies & Car Showrooms,South Governorate,Sidon District
3,Al Zarif Exhibition,03-404785,Vehicles & Transport,Agencies & Car Showrooms,South Governorate,Sidon District
4,Al-Ghali Motors,03-710705,Vehicles & Transport,Agencies & Car Showrooms,Beirut & Mount Lebanon,Cola


In [10]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('shops.csv')
df.drop(columns=['sub_subcategory'], inplace=True)


# Clean whitespace and convert to lowercase for comparison
df['subcategory'] = df['subcategory'].str.strip().str.lower()
df['location'] = df['location'].str.strip().str.lower()

# Print available subcategories and locations
available_subcategories = df['subcategory'].unique()
available_locations = df['location'].unique()

print("Available Subcategories:")
print(available_subcategories)
print("\nAvailable Locations:")
print(available_locations)

# Get input from the user for subcategory and location
user_subcategory = input("Enter the subcategory you are interested in: ").strip().lower()
user_location = input("Enter the location you are interested in: ").strip().lower()

# Filter the DataFrame based on user input
filtered_df = df[(df['subcategory'] == user_subcategory) & (df['location'] == user_location)]

# Print the head of the filtered DataFrame
print("\nHead of Filtered DataFrame:")
print(filtered_df.head())


Available Subcategories:
['agencies & car showrooms' 'car rental' 'motorcycles & bikes'
 'translation, proofreading and completion of transactions'
 'tires & car spare parts' 'automotive accessories' 'car maintenance'
 'online shop' 'fuel & sell & change oil' 'maintenance companies'
 'craftsmen' 'installation & maintenance' 'driving schools'
 'service stations' 'industrial equipments & engines'
 'traffic accident experts' 'transport & taxi companies'
 'travel & tourism agencies' 'shipping & customs clearance'
 'engineers & engineering companies' 'building contractors'
 'furniture - carpets - linens - curtains - decor'
 'cleaning and sterilization services' 'construction & building materials'
 'import & export' 'home & electrical appliances' 'factories' 'trading'
 'perfumes - gifts - accessories' 'hvac & plumbing & solar energy'
 'electrical materials' 'arts & graphic design'
 'preparation (offices, hotels, restaurants, supermarkets)' 'industry'
 'real estate companies' 'antiques & hand

Enter the subcategory you are interested in:  agencies & car showrooms
Enter the location you are interested in:  sidon district



Head of Filtered DataFrame:
                shop_name phone_number               category  \
0     Abdullah Exhibition    03-773813  Vehicles & Transport    
2         Al Haytham Expo    71-582907  Vehicles & Transport    
3     Al Zarif Exhibition    03-404785  Vehicles & Transport    
21   Kajak & Mat Showroom    70-461400  Vehicles & Transport    
31  Modern Car Exhibition    70-105777  Vehicles & Transport    

                 subcategory               city        location  
0   agencies & car showrooms  South Governorate  sidon district  
2   agencies & car showrooms  South Governorate  sidon district  
3   agencies & car showrooms  South Governorate  sidon district  
21  agencies & car showrooms  South Governorate  sidon district  
31  agencies & car showrooms  South Governorate  sidon district  
